In [302]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp
from sympy.solvers import solve
import pandas as pd

## Модельная задача

### Аналитическое решение

In [303]:
k = 1.25
q = 2.5
f = math.cos(0.5)
beta1 = 100; eps1 = 0
beta2 = 0; eps2 = 0

In [304]:
def analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2):
    from sympy.abc import a,b
    lmb = lambda q,k: math.sqrt(q/k)
    u = lambda c1,c2,lmb,x,f,q: c1*math.exp(lmb*x) + c2*math.exp(-lmb*x) + f/q

    lmb1 = lmb(q,k); lmb2 = - lmb1
    equations = [(k*lmb1-beta1)*a+(k*lmb2-beta1)*b-beta1*f/q+eps1,
                 -(k*lmb1-beta2)*math.exp(lmb1)*a-(k*lmb2+beta2)*math.exp(lmb2)*b-beta2*f/q+eps2]
    constants = solve(equations,[a,b],dict=False)
    c1 = constants[a]; c2 = constants[b]
    u1 = u(c1,c2,lmb1,x,f,q)
    return u1

In [305]:
def analytic_points(k,q,f,beta1,eps1,beta2,eps2):
        x1 = np.linspace(0,1,11)
        y1 = []; y2 = []
        for x in x1:
                res1 = analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2)
                y1.append(res1)
        return y1

### Метод прогонки

In [306]:
def progonka_1(n,k,q,f,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1
    solutions = np.zeros(11)
    u = np.zeros(n+1); alpha = np.zeros(n+1); beta = np.zeros(n+1)
    al = k; bl = -2*k - q*h**2; cl = k; dl = -f*h**2
    a0 = k; b0 = -k-beta1*h; c0 = 0; d0 = -eps1*h
    aL = 0; bL = -k-beta2*h; cL=k; dL = -eps2*h
    
    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al/(bl+cl*alpha[i-1])
        beta[i] = (dl - cl*beta[i-1])/(bl+cl*alpha[i-1])
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]; count = 9; t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [307]:
x = np.linspace(0,1,11)
u1 = progonka_1(10,k,q,f,beta1,eps1,beta2,eps2)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
df = pd.DataFrame({"x": x, "u аналит.": a1, "u числ. ": u1 ,"u числ.- u аналит. ": delta1})
display(df)
print("Максимальная норма ошибки =", max(delta1))

,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0,0.00542759039477941,0.004918,0.000510070268226847
1,0.1,0.0455313758259506,0.044258,0.00127369468697747
2,0.2,0.0795149380988694,0.077462,0.00205260281981928
3,0.3,0.108059081999863,0.105196,0.00286350637027830
4,0.4,0.131735642512957,0.128012,0.00372357551536792
5,0.5,0.151018940593429,0.146368,0.00465080138300739
6,0.6,0.166295285408163,0.160631,0.00566437169582312
7,0.7,0.177870713404542,0.171086,0.00678506741116081
8,0.8,0.185977119247418,0.177941,0.00803568854825071
9,0.9,0.190776901447384,0.181335,0.00944151792356940


Максимальная норма ошибки = 0.0110308322240632


## Задача с переменными коеффициентами

In [308]:
def progonka_2(n,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1; x = 0 + h
    solutions = np.zeros(11)
    k = lambda x: x**2 + 1
    q = lambda x: x + 2
    f = lambda x: math.cos(x)
    
    u = np.zeros(n+1); alpha = np.zeros(n+1); beta = np.zeros(n+1)
    al = lambda x: k(x+h/2); bl = lambda x: -(k(x+h/2)+k(x-h/2)+q(x)*h**2); cl = lambda x: k(x-h/2); dl = lambda x: -f(x)*h**2
    a0 = k(0); b0 = -k(0)-beta1*h; c0 = 0; d0 = -eps1*h
    aL = 0; bL = -k(1)-beta2*h; cL=k(1); dL = -eps2*h

    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al(x)/(bl(x)+cl(x)*alpha[i-1])
        beta[i] = (dl(x) - cl(x)*beta[i-1])/(bl(x)+cl(x)*alpha[i-1])
        x += h
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]; count = 9; t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [327]:
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [328]:
x = np.linspace(0,1,11)
u2 = progonka_2(30,beta1,eps1,beta2,eps2)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"u числ.- u аналит. ": delta2})
display(df)
print("Максимальная норма ошибки =", max(delta2))

,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0,0.005093,0.004927,0.000166
1,0.1,0.051094,0.050917,0.000177
2,0.2,0.087535,0.087341,0.000195
3,0.3,0.115208,0.114991,0.000217
4,0.4,0.135260,0.135016,0.000243
5,0.5,0.149021,0.148748,0.000273
6,0.6,0.157849,0.157543,0.000306
7,0.7,0.163013,0.162672,0.000342
8,0.8,0.165634,0.165254,0.000380
9,0.9,0.166656,0.166236,0.000420


Максимальная норма ошибки = 0.00046284328667042796


## Таблица результатов

In [329]:
x = np.linspace(0,1,11)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
a2 = progonka_2(10**6,beta1,eps1,beta2,eps2)

In [331]:
delta1 = []; delta2 = []; 
u1 = progonka_1(100,k,q,f,beta1,eps1,beta2,eps2)
u2 = progonka_2(100,beta1,eps1,beta2,eps2)
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": u1 ,"u мод. числ.- u мод. аналит. ": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": u2 ,"u пер. числ.- u пер. аналит. ": delta2})
display(df)
print("Максимальная норма ошибки модельной задачи =", max(delta1))
print("Максимальная норма ошибки реальной задачи = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,u мод. числ.- u мод. аналит.,u пер. аналит.,u пер. числ.,u пер. числ.- u пер. аналит.
0,0.0,0.00542759039477941,0.005376,5.14432538044271e-5,0.005093,0.005043,0.000050
1,0.1,0.0455313758259506,0.045412,0.000119462085652756,0.051094,0.051042,0.000052
2,0.2,0.0795149380988694,0.079325,0.000189771933045813,0.087535,0.087480,0.000055
3,0.3,0.108059081999863,0.107795,0.000263792679990243,0.115208,0.115148,0.000060
4,0.4,0.131735642512957,0.131393,0.000343016731881335,0.135260,0.135194,0.000066
5,0.5,0.151018940593429,0.150590,0.000429039104133611,0.149021,0.148949,0.000073
6,0.6,0.166295285408163,0.165772,0.000523589533788948,0.157849,0.157769,0.000080
7,0.7,0.177870713404542,0.177242,0.000628567260571589,0.163013,0.162925,0.000089
8,0.8,0.185977119247418,0.185231,0.000746079176747488,0.165634,0.165536,0.000098
9,0.9,0.190776901447384,0.189898,0.000878482112094764,0.166656,0.166549,0.000108


Максимальная норма ошибки модельной задачи = 0.00102843010263720
Максимальная норма ошибки реальной задачи =  0.00011813563889839784
